# Lab 1


## Part 1: Bilingual dictionary induction and unsupervised embedding-based MT (30%)
*Note: this homework is based on materials from yandexdataschool [NLP course](https://github.com/yandexdataschool/nlp_course/). Feel free to check this awesome course if you wish to dig deeper.*

*Refined by [Nikolay Karpachev](https://www.linkedin.com/in/nikolay-karpachev-b0146a104/)*

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully), in particular for similar languages, e.g. Ukrainian and Russian. 

### Frament of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our for our purposes.

## Data

In this notebook we're going to use pretrained word vectors - FastText (original paper - https://arxiv.org/abs/1607.04606).

You can download them from the official [website](https://fasttext.cc/docs/en/crawl-vectors.html). We're going to need embeddings for Russian and Ukrainian languages.

In [1]:
!wget -nc https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.vec.gz
!gzip -d cc.ru.300.vec.gz

!wget -nc https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.uk.300.vec.gz
!gzip -d cc.uk.300.vec.gz

--2022-04-08 11:25:25--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1306357571 (1.2G) [binary/octet-stream]
Saving to: ‘cc.ru.300.vec.gz’

cc.ru.300.vec.gz    100%[===================>]   1.22G  50.0MB/s    in 25s     

2022-04-08 11:25:51 (49.0 MB/s) - ‘cc.ru.300.vec.gz’ saved [1306357571/1306357571]

--2022-04-08 11:26:31--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.uk.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1257595219 (1.2G) [binary/octet-stream]
Saving to: ‘cc.uk.300

After downloading and extracting the vectors, we should be able to load them using the [gensim](https://radimrehurek.com/gensim/) library:

In [2]:
from gensim.models import KeyedVectors


uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")

Once you've loaded the vectors, you can use the `KeyedVectors` interface to get word embeddings and/or query most similar words by embedding:

In [3]:
august_embedding = ru_emb["август"]
august_embedding.shape, august_embedding[:5]

((300,), array([ 0.0033, -0.0322, -0.0519, -0.0808, -0.0131], dtype=float32))

In [4]:
ru_emb.most_similar([august_embedding])

[('август', 1.0),
 ('июль', 0.9383153915405273),
 ('сентябрь', 0.9240028858184814),
 ('июнь', 0.9222575426101685),
 ('октябрь', 0.9095538854598999),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545796275138855),
 ('февраль', 0.8401416540145874)]

The latter function also allows you to vary the amount of closest words via the `topn` argument:

In [5]:
ru_emb.most_similar([august_embedding], topn=3)

[('август', 1.0),
 ('июль', 0.9383153915405273),
 ('сентябрь', 0.9240028858184814)]

Another feature of `KeyedVectors` is that it allows to compute embeddings for multiple words simultaneously:

In [6]:
ru_emb[["август", "сентябрь"]].shape

(2, 300)

Everything above is true for the embeddings for Ukrainian language.

In [7]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999999403953552),
 ('липень', 0.9096440076828003),
 ('вересень', 0.901697039604187),
 ('червень', 0.8992519378662109),
 ('жовтень', 0.8810408711433411),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.8408110737800598),
 ('лютий', 0.8256431818008423)]

However, russian and ukrainian embeddings were trained independently of each other. This means, that there is no obvious connection between values in embeddings for similar words in Russian and Ukrainian:

In [8]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Stepashka.com', 0.2757962942123413),
 ('ЖИЗНИВадим', 0.25203436613082886),
 ('2Дмитрий', 0.25048112869262695),
 ('2012Дмитрий', 0.24829231202602386),
 ('Ведущий-Алексей', 0.2443869560956955),
 ('Недопустимость', 0.24435284733772278),
 ('2Михаил', 0.23981399834156036),
 ('лексей', 0.23740756511688232),
 ('комплексн', 0.23695150017738342),
 ('персональ', 0.2368222028017044)]

## Translation

We'll build a simple translator, which will try to predict the russian embedding from the ukrainian one. For this we'll need a dataset of word pairs.

In [9]:
import pandas as pd


train_data_url = "https://raw.githubusercontent.com/girafe-ai/natural-language-processing/22s_made/homeworks/lab01_unsupervised_translation/uk_ru.train.tsv"
train_data = pd.read_csv(train_data_url, sep="\t", header=None)
train_data.columns = ["uk", "ru"]
print(f"Train dataset size: {len(train_data)}")

test_data_url = "https://raw.githubusercontent.com/girafe-ai/natural-language-processing/22s_made/homeworks/lab01_unsupervised_translation/uk_ru.test.tsv"
test_data = pd.read_csv(test_data_url, sep="\t", header=None)
test_data.columns = ["uk", "ru"]
print(f"Test dataset size: {len(test_data)}")

train_data.head()

Train dataset size: 1927
Test dataset size: 400


,uk,ru
0,iмовірно,вероятно
1,iснує,существует
2,iспит,экзамен
3,абияк,как-нибудь
4,або,или


Our method won't work with unknown words, so let's filter them out:

In [10]:
rows = []
for _, row in train_data.iterrows():
    if row["uk"] not in uk_emb or row["ru"] not in ru_emb:
        continue

    rows.append(row)

train_data = pd.DataFrame(rows)
print(f"Train dataset size: {len(train_data)}")

rows = []
for _, row in test_data.iterrows():
    if row["uk"] not in uk_emb or row["ru"] not in ru_emb:
        continue

    rows.append(row)

test_data = pd.DataFrame(rows)
print(f"Test dataset size: {len(test_data)}")

Train dataset size: 1880
Test dataset size: 393


We will train our model to predict embedding for the russian word from embedding of its ukrainian counterpart. For this reason we split our train and test data into ukrainian and russian words and compute corresponding embeddings to obtain `X` (ukrainian embeddings) and `y` (russian embeddings).

In [11]:
X_train, Y_train = uk_emb[train_data["uk"].values], ru_emb[train_data["ru"].values]
X_test, Y_test = uk_emb[test_data["uk"].values], ru_emb[test_data["ru"].values]

## Embedding space mapping (0.3 pts)

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n\|Wx_i - y_i\|_2$$

or

$$W^*= \arg\min_W \|XW^T - Y\|_F$$

where $\|\cdot\|_F$ denotes Frobenius norm.

> **Note:** in second formula, $W$ and $x$ seem to have switched places. This happens because the $X$ matrix is composed of objects $x_i$ in *rows* not *columns*, i.e. it is kind of composed of $x_i^T$. This means that $X \in \mathbb{R}^{N \times D}$, where $N$ is the number of items and $D$ is the embedding dimensionality. The same is true for the $Y$.

$W^*= \arg\min_W \sum_{i=1}^n\|Wx_i - y_i\|_2$ looks like simple multiple linear regression without bias. The `sklearn` allows you to turn off the bias in `LinearRegression` via the `fit_intercept` argument (in fact they simply call bias the intercept). So let's code.

In [12]:
from sklearn.linear_model import LinearRegression


# YOUR CODE HERE
mapping = LinearRegression(fit_intercept=False).fit(X_train, Y_train)

Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [13]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8531432747840881),
 ('июнь', 0.8402522802352905),
 ('март', 0.8385884165763855),
 ('сентябрь', 0.8331484794616699),
 ('февраль', 0.8311208486557007),
 ('октябрь', 0.8278019428253174),
 ('ноябрь', 0.8243728280067444),
 ('июль', 0.8229618072509766),
 ('август', 0.8112280368804932),
 ('январь', 0.8022986650466919)]

We can see that neighbourhood of this embedding cosists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in russian embedding space).

In [14]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    total = len(pairs)
    correct = 0
    for i in range(total):
        pair = pairs[i]
        predicted_vector = mapped_vectors[i]

        # YOUR CODE HERE
        for word, _ in ru_emb.most_similar([mapped_vectors[i]], topn=topn):
            if pairs[i][1] == word:
                correct += 1

    return correct / total

In [15]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

Note that our `precision` function accepts lists of pairs of words, whereas we have dataframes. However, it is not a problem: we can get a list (actually, numpy array) of pairs via the `values` property.

In [16]:
assert precision(test_data.values, X_test) == 0.0
assert precision(test_data.values, Y_test) == 1.0

Let's see how well our model is doing.

In [17]:
top1 = precision(test_data.values, mapping.predict(X_test), 1)
print(f"Top-1 precision {100 * top1:.1f}%")

top5 = precision(test_data.values, mapping.predict(X_test), 5)
print(f"Top-5 precision {100 * top5:.1f}%")

Top-1 precision 62.8%
Top-5 precision 79.1%


## Making it better (orthogonal Procrustean problem) (0.3 pts)

It can be shown that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$(W^T)^*= \arg\min_{W^T} \|XW^T - Y\|_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$(W^T)^*=UV^T$$

In [18]:
import numpy as np


# YOUR CODE HERE
# Compute the orthogonal mapping (W^T)^* as defined in formula above.
u, s, vt = np.linalg.svd(X_train.T @ Y_train)
mapping = u @ vt

Now our `mapping` is just a numpy array, meaning that it has no `predict` method. However, from the formulae above we know, that prediction is done using the matrix multiplication:

In [19]:
august = uk_emb["серпень"] @ mapping
ru_emb.most_similar([august])

[('апрель', 0.8245131969451904),
 ('июнь', 0.805662989616394),
 ('сентябрь', 0.8055761456489563),
 ('март', 0.8032935261726379),
 ('октябрь', 0.7987102270126343),
 ('июль', 0.7946797013282776),
 ('ноябрь', 0.7939636707305908),
 ('август', 0.7938189506530762),
 ('февраль', 0.7923861145973206),
 ('декабрь', 0.7715375423431396)]

Now let's compute our precision values and see, whether our trick did improve the results.

In [20]:
top1 = precision(test_data.values, X_test @ mapping, 1)
print(f"Top-1 precision {100 * top1:.1f}%")

top5 = precision(test_data.values, X_test @ mapping, 5)
print(f"Top-5 precision {100 * top5:.1f}%")

Top-1 precision 64.4%
Top-5 precision 79.9%


## Unsupervised embedding-based MT (0.4 pts)

Now, let's build our word embeddings-based translator!

Firstly, download OPUS Tatoeba corpus.

In [21]:
!wget https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz
!gzip -d ./uk.txt.gz

--2022-04-08 11:44:30--  https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1819128 (1.7M) [application/gzip]
Saving to: ‘uk.txt.gz’

uk.txt.gz           100%[===================>]   1.73M  2.29MB/s    in 0.8s    

2022-04-08 11:44:32 (2.29 MB/s) - ‘uk.txt.gz’ saved [1819128/1819128]



In [22]:
with open('./uk.txt', 'r') as f:
    uk_corpus = f.readlines()

# To save your time and CPU, feel free to use first 1000 sentences of the corpus
uk_corpus = uk_corpus[:1000]
uk_corpus[:5]

['Я вже закінчу коледж, коли ви вернетеся з Америки.\n',
 'Він наказав мені негайно вийти з кімнати.\n',
 'Як би ти не намагався, ти не вивчиш англійську за два-три місяці.\n',
 'Поки я не подзвонив, він не прийшов.\n',
 'У всесвіті багато галактик.\n']

Now let's translate these sentences word-by-word. Before that, however, don't forget to tokenize your sentences. For that you may (or may not) find the `nltk.tokenize.WordPunctTokenizer` to be very useful.

In [23]:
from nltk.tokenize import WordPunctTokenizer

def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    translated = []

    # YOUR CODE HERE
    tokenizer = WordPunctTokenizer()

    for uk_word in tokenizer.tokenize(sentence):
        if uk_word in uk_emb:
            ru_word, _ = ru_emb.most_similar([uk_emb[uk_word] @ mapping], topn=1)[0]
            translated.append(ru_word)
        else:
            translated.append(uk_word)

    return " ".join(translated)

In [24]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

Now you can play with your model and try to get as accurate translations as possible. **Note**: one big issue is out-of-vocabulary words. Try to think of various ways of handling it (you can start with translating each of them to a special **UNK** token and then move to more sophisticated approaches). Good luck!

In [25]:
for sent in uk_corpus[::10]:
    print(translate(sent))

Я уже закончу колледж , когда мы прибежишь со Америки .
Город бомбили враждебные самолеты .
Возможно , мной антисоциальный , конечно это не означает , что мной не общаюсь со людьми .
Впрочем утра выпала роса .
Беда не приходит одна .
Посмотри по тот дым .
Я заказал два гамбургера .
Я не хотел никого обидеть .
Гора покрыта снегом .
по фотографии во девушки корона не со золота , а со цветов .
Во меня То мечта .
Я приехал во Японию со Китая .
по север находится Шотландия ; по юге — Англия ; по востоке — Уэльс ; и ещe дальше по востоке — северная Ирландия .
Его родная страна — Германия .
Берн — столица Швейцарии .
Он ждал по него к десятой часа .
Ты можешь взять ту книгу даром .
Такой роман сочинил известный американский писатель .
забронировать , будте ласковые , комнату возле международного аэропорта во Торонто .
Он знает , что ты его влюбится ?
Я знаю , что ты богатый .
Те , кто всё забывают , счастливые .
Во этой реке опасно плавать .
пришел , увидел , победил .
Я хожу к школы пешком .

Let's try to use a [stemmer for the Ukrainian language](https://github.com/Desklop/Uk_Stemmer) (this may reduce the number of words that the model cannot translate, but the translation itself may suffer, because most likely all words will be translated in their initial form

In [26]:
!pip install git+https://github.com/Desklop/Uk_Stemmer

  Cloning https://github.com/Desklop/Uk_Stemmer to /tmp/pip-req-build-nanyua1l
  Running command git clone -q https://github.com/Desklop/Uk_Stemmer /tmp/pip-req-build-nanyua1l
  Created wheel for uk-stemmer: filename=uk_stemmer-1.0-py3-none-any.whl size=9511 sha256=eb99ecbaafae21561c3ee626ead0f88b37ffe21298a60c71737d3b67a6a15669
  Stored in directory: /tmp/pip-ephem-wheel-cache-29racx6b/wheels/40/53/2f/982431dc2aff61a4eb52a030950bbbdef0d9178114373d6c0e
Successfully built uk-stemmer


In [27]:
import re
from uk_stemmer import UkStemmer

stemmer = UkStemmer()

test_string = 'Привіт, як твої справи? Зберігайте спокойствіе. Будь ласка, зберігайте спокій.'

prepare_test_string = test_string.lower()
words = re.split(r'(\W)', prepare_test_string)
words = [word for word in words if word != '']

for i in range(len(words)):
    words[i] = stemmer.stem_word(words[i])

stem_test_string = ''.join(words)

print(stem_test_string)

привіт, як тво справ? зберігайт спокойств. буд ласк, зберігайт спок.


In [28]:
def translate_with_stem(sentence):

    stemmer = UkStemmer()

    prepare_sentence = sentence.lower()
    words = re.split(r'(\W)', prepare_sentence)
    words = [word for word in words if word != '']

    for i in range(len(words)):
        words[i] = stemmer.stem_word(words[i])

    translated = []
    
    for uk_word in words:
        if uk_word in uk_emb:
            ru_word, _ = ru_emb.most_similar([uk_emb[uk_word] @ mapping], topn=1)[0]
            translated.append(ru_word)
        else:
            translated.append(uk_word)

    return re.sub(r'\s+', ' ', " ".join(translated))

In [29]:
for sent in uk_corpus[::10]:
    print(f'UK: \n {sent.strip()}')
    print(f'RU (with stem): \n {translate_with_stem(sent)}')
    print(f'RU (without stem): \n {translate(sent)} \n')

UK: 
 Я вже закінчу коледж, коли ви вернетеся з Америки.
RU (with stem): 
 мной уже закончил колледж , Желтухинский мы возвратишь со океании . 
RU (without stem): 
 Я уже закончу колледж , когда мы прибежишь со Америки . 

UK: 
 Місто бомбардували ворожі літаки.
RU (with stem): 
 городов бомбардув ворож самолет . 
RU (without stem): 
 Город бомбили враждебные самолеты . 

UK: 
 Можливо, я антисоціальний, але це не означає, що я не спілкуюся з людьми.
RU (with stem): 
 возможност , мной антисоціальн , ым это не означить , что мной не программаДля со людьми . 
RU (without stem): 
 Возможно , мной антисоциальный , конечно это не означает , что мной не общаюсь со людьми . 

UK: 
 Цього ранку випала роса.
RU (with stem): 
 этого.Если дартсмен обжиг анг . 
RU (without stem): 
 Впрочем утра выпала роса . 

UK: 
 Біда не приходить одна.
RU (with stem): 
 бед не прихожу мын . 
RU (without stem): 
 Беда не приходит одна . 

UK: 
 Подивися на той дим.
RU (with stem): 
 удивление по то дым . 
RU (

The results got worse, but it was worth a try)